In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import Colorblind
from pyttoresque import netlist, simserver
output_notebook()

In [ ]:
# Configuration
name = "top$top"
outputs = ["in", "out"]
tstep = 3e-6
tstop = 1e-3
tstart = 0

In [ ]:
m = netlist.Modeldict()
seq, schem = netlist.get_all_schem_docs(name)
spice = netlist.spice_netlist(name, schem, m)
print(spice)

In [ ]:
sim = simserver.connect("vps.pepijndevos.nl", 9418)
fileset = sim.loadFiles([{"name": "mosaic.sp", "contents": spice}])
res = fileset.commands.tran(tstep, tstop, tstart)
tran = simserver.readAll(res)
plot = figure(title="my sine wave", output_backend="webgl")

for  o, c in zip(outputs, Colorblind[8]):
    plot.line(tran.scale, o, source=tran.data, color=c)

target = show(plot, notebook_handle=True)

In [ ]:
def update(schem):
    print("update!")
    tran.data.data = {k:[] for k in tran.data.column_names}
    push_notebook(handle=target)
    spice = netlist.spice_netlist(name, schem, m)
    fileset = sim.loadFiles([{"name": "mosaic.sp", "contents": spice}])
    res = fileset.commands.tran(3e-6, 1e-2, 0)
    simserver.stream(res, tran.data, cell=target)

netlist.live_schem_docs(name, update)